## Modules

In [18]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np

## Config

In [19]:
INCLUDE_IMPOSSIBLE=False

## Process data

In [20]:
pd.set_option('display.max_columns', None)
hun_data = pd.read_csv("data/dev_long_impossible_yes_no_arithmetic.csv", sep=';')
#192. sor hiba!

In [21]:
hun_data = hun_data.drop(hun_data.columns[0], axis=1) # drop nameless column
hun_data.head()

,id,user,context,title,section,is_impossible,question,end,start,answer,type,modanswer
0,1578,5,A török építészet formavilágának alapjai a nép...,Törökország,Építészet,True,Mikor épült az isztambuli Kék mecset?,0,0,NaN,NaN,NaN
1,3122,3,Puskás Ferenc apai ágon szegény sváb származás...,Puskás Ferenc (labdarúgó),"Gyermekkor, Kispesti AC",False,Miért nem a születése napján ünnepelte Puskás ...,256,0,Puskás Ferenc apai ágon szegény sváb származás...,[],NaN
2,2768,4,"Watson 2005-ben a Teen Vogue-ban, egy a kifeje...",Emma Watson,Modellkedés és divat,False,Melyik cég arca lett Emma Watson 2011 márciusá...,666,522,2011 januárjában Watson elnyerte az Elle Style...,[],NaN
3,1526,5,A kritikusok folyamatosan elemzik és vizsgáljá...,Lady Gaga,Zenei stílusa és témái,False,Miért változtatja folyton Gaga a zenei stílusát?,210,0,A kritikusok folyamatosan elemzik és vizsgáljá...,[],NaN
4,290,6,A hadihajók tüze után a szövetséges légierő ha...,Normandiai partraszállás,NaN,True,Melyik irányba tértek el a szövetséges bombázó...,0,0,NaN,NaN,NaN


In [22]:
hun_data.dtypes

id                int64
user              int64
context          object
title            object
section          object
is_impossible      bool
question         object
end               int64
start             int64
answer           object
type             object
modanswer        object
dtype: object

In [23]:
impossible_answer = "Nincs elegendő adat a kérdés megválaszolásához."

if INCLUDE_IMPOSSIBLE:
    prompt_prefix = ("Válaszold meg az alábbi kérdést a forrás alapján! "
                     "Ha a forrás alapján nem megválaszolható a kérdés, mondd hogy: "
                     f"{impossible_answer}")
else:
    prompt_prefix = "Válaszold meg az alábbi kérdést a forrás alapján!"

print(f"{prompt_prefix=}")
prompt_col = []
impossible_question_count = 0

with tqdm(total=len(hun_data.index), desc="Processing data") as pbar:
    for index, row in hun_data.loc[:].iterrows():

        if row["answer"] is np.nan:
            hun_data.loc[index, "is_impossible"] = True
        if hun_data.loc[index, "is_impossible"]:
            hun_data.loc[index, "answer"] = impossible_answer
            impossible_question_count = impossible_question_count + 1
        if row["modanswer"] is not np.nan:
            hun_data.loc[index, "answer"] = row["modanswer"]

        hun_data.loc[index, "answer"] = hun_data.loc[index, "answer"].strip(r" []'")

        text = (prompt_prefix
                + "\n### Forrás:\n" + str(row['context'])
                + "\n### Kérdés:\n" + str(row['question'])
                + "\n### Válasz:\n" + str(hun_data.loc[index, "answer"]))

        text = text.strip()
        prompt_col.append(text)
        pbar.update(1)
        
hun_data["text"] = prompt_col
print(f"{impossible_question_count=}")

prompt_prefix='Válaszold meg az alábbi kérdést a forrás alapján!'


Processing data: 100%|████████████████████████████████████████████████████████████| 4553/4553 [00:01<00:00, 3003.15it/s]

impossible_question_count=1335


In [24]:
print(f"{len(hun_data)=}")
if not INCLUDE_IMPOSSIBLE:
    hun_data = hun_data[hun_data.answer != impossible_answer]
    print(f"New {len(hun_data)=}")

len(hun_data)=4553
New len(hun_data)=3218


## Create train and test sets

In [25]:
print(f"{hun_data.shape=}")
hun_data.tail()

hun_data.shape=(3218, 13)


,id,user,context,title,section,is_impossible,question,end,start,answer,type,modanswer,text
4544,2077,5,Bécsben az Artaria kiadóvállalat tett sokat Ha...,Joseph Haydn,Haydn növekvő hírneve,False,Ki adta ki Bécsben Haydn műveit?,74,0,Bécsben az Artaria kiadóvállalat tett sokat Ha...,[],NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4546,3191,3,A viselési kényelem fokozása érdekében az egés...,Egészségügyi harisnya,Az egészségügyi harisnyák gyártása,False,Mikor tartósabb az egészségügyi harisnya anató...,898,373,A harisnya anatómiai formáját vagy a kötés sor...,[],NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4547,1400,5,A hagyomány szerint már Mohamed felszólította ...,Iszlám,Az iszlám elterjedése,False,Milyen nehézségekkel kellett szembenézniük a m...,554,246,Mohamed halálakor a közösség kettős kihívás el...,[],NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4548,1491,5,Mozart ekkor végleg szakított a salzburgi érse...,Wolfgang Amadeus Mozart,NaN,False,Hol mutatták be elsőként a Don Giovanni című o...,1200,1138,1787-ben a Don Giovannit óriási sikerrel mutat...,[],NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4550,694,4,Az Hagia Szophia erőteljes szimbóluma a Görögk...,Hagia Szophia,Hagia Szophia – vallás és politika,False,Mire bizonyíték az Hagia Szophia Chris Spirou ...,772,588,Spirou szerint: „Az Hagia Szophia a legnagyobb...,[],NaN,Válaszold meg az alábbi kérdést a forrás alapj...


In [26]:
hun_data_train, hun_data_temp = train_test_split(hun_data, test_size=0.3, random_state=42)
hun_data_eval, hun_data_test = train_test_split(hun_data_temp, test_size=0.5, random_state=42)

In [27]:
print("hun_data_train shape:", hun_data_train.shape)
print("hun_data_test shape:", hun_data_test.shape)
print("hun_data_eval shape:", hun_data_eval.shape)

hun_data_train shape: (2252, 13)
hun_data_test shape: (483, 13)
hun_data_eval shape: (483, 13)


In [28]:
hun_data_train_q_c_a_t = hun_data_train[['question','context','answer', 'text']]
hun_data_test_q_c_a_t = hun_data_test[['question','context','answer', 'text']]
hun_data_eval_q_c_a_t = hun_data_eval[['question','context','answer', 'text']]

In [29]:
hun_data_train_q_c_a_t.head()

,question,context,answer,text
1894,Mi a kapcsolat Pannonhalma és Habsburg Ottó kö...,A főhajó két szélén nyíló kapukon át megközelí...,Az altemplomban helyezték el Habsburg Ottó szí...,Válaszold meg az alábbi kérdést a forrás alapj...
3907,Mit jelent a falkár szó?,"Trénere az Angliából Magyarországra, Gödre köl...","Trénere az Angliából Magyarországra, Gödre köl...",Válaszold meg az alábbi kérdést a forrás alapj...
1418,Haydnnak mennyi volt az éves fizetése Karl Jos...,Haydn tanítványainak a száma látványosan megug...,1759-ben von Fürnberg beajánlotta Karl Joseph ...,Válaszold meg az alábbi kérdést a forrás alapj...
3750,Miből áll a Mars légköre?,"Légköre igen ritka, a felszíni légnyomás a föl...","Légköre igen ritka, a felszíni légnyomás a föl...",Válaszold meg az alábbi kérdést a forrás alapj...
3545,Mi az a kemény indulás?,Kurzweil a nézeteit könyveiben: A spirituális ...,"Jóslatai eltérnek Vinge-étől, mivel szerinte a...",Válaszold meg az alábbi kérdést a forrás alapj...


In [30]:
hun_data_test_q_c_a_t.head()

,question,context,answer,text
4323,Mit jelent a női körülmetélés?,A női körülmetélés (vagyis a női nemi szervek ...,A női körülmetélés (vagyis a női nemi szervek ...,Válaszold meg az alábbi kérdést a forrás alapj...
2487,Mikor indult meg a lóvasút Budán?,Budapest villamosvonal-hálózata a magyar fővár...,Budán csak 1868. május 18-án avatták fel az el...,Válaszold meg az alábbi kérdést a forrás alapj...
300,Volt testvére Orseolo Péternek?,"Apja testvérének fiai, azaz István unokatestvé...",Választása végül Ilona nevű lánytestvérének fi...,Válaszold meg az alábbi kérdést a forrás alapj...
1116,A gleccserek édes vagy sós vizet tartalmaznak?,A globális felmelegedés következtében a gleccs...,A globális felmelegedés következtében a gleccs...,Válaszold meg az alábbi kérdést a forrás alapj...
4326,Min aludtak az ókori egyiptomiak?,Az átlagos egyiptomi lakóépületek a falvakban ...,"Szalmazsákon aludtak, párna helyett kőből vagy...",Válaszold meg az alábbi kérdést a forrás alapj...


In [31]:
hun_data_eval_q_c_a_t.head()

,question,context,answer,text
466,Igaznak tartotte-a VI. Kelemen a zsidókat ért ...,"1348-ban Európa-szerte elterjedt az a rémhír, ...",Összesen legalább kétszáz városban voltak ilye...,Válaszold meg az alábbi kérdést a forrás alapj...
436,Milyen tulajdonságai voltak Irinyi biztonsági ...,A zajtalan és robbanásmentes biztonsági gyufát...,A zajtalan és robbanásmentes biztonsági gyufát...,Válaszold meg az alábbi kérdést a forrás alapj...
4029,Hogy nevezték az orosz szociálforradalmárokat?,A párt és a mozgalom népszerűsége rohamosan nő...,Miután az 1917-es választást az eszerek (szoci...,Válaszold meg az alábbi kérdést a forrás alapj...
1787,Hol van a Naprendszer vége?,A Naprendszer a Nap gravitációja által egyben ...,"A tudósok csak a XX. században jöttek rá, hogy...",Válaszold meg az alábbi kérdést a forrás alapj...
1742,Ki írta le először az abszintizmus tüneteit?,Dr. Valentin Magnan készített tanulmányt elsők...,Dr. Valentin Magnan készített tanulmányt elsők...,Válaszold meg az alábbi kérdést a forrás alapj...


In [32]:
hun_data_train_q_c_a_t.to_csv("data/train.csv", index=False, sep=";")
hun_data_test_q_c_a_t.to_csv("data/test.csv", index=False, sep=";")
hun_data_eval_q_c_a_t.to_csv("data/eval.csv", index=False, sep=";")